<a href="https://colab.research.google.com/github/psy794/stock_pjtt/blob/master/LR_0712_(%EA%B2%B0%EA%B3%BC_0_38082)_%EA%B3%B5%EC%9C%A0%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
!pip install yfinance
!pip install finance-datareader
!pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 23.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import random
import os
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import pickle

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

#증권시장의 주식정보 스크래핑
#pip install pykrx
from pykrx import stock
from pykrx import bond

import yfinance as yf
import FinanceDataReader as fdr
import pykrx

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Data Load

In [ ]:
# 저장된 pickle 파일 불러오기
loaded_train = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/train_data.pkl', 'rb'))

In [ ]:
train = loaded_train
train

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드
0,2021-06-01,2890,2970,2885,2920,166690,487455970,1.040000,060310
1,2021-06-02,2915,2975,2830,2900,134833,388795625,-0.680000,060310
2,2021-06-03,2900,2925,2875,2900,144470,419668300,0.000000,060310
3,2021-06-04,2930,3120,2920,2950,934224,2840373820,1.720000,060310
4,2021-06-07,3000,3150,2955,3150,946560,2929678225,6.780000,060310
...,...,...,...,...,...,...,...,...,...
987995,2023-05-23,6710,6710,6420,6430,40981,266207710,-4.170000,238490
987996,2023-05-24,6400,6490,6210,6320,19392,123635550,-1.710000,238490
987997,2023-05-25,6340,6400,6270,6330,6457,40862130,0.160000,238490
987998,2023-05-26,6330,6410,6300,6330,8905,56621540,0.000000,238490


# 파이프라인 (120기간으로 15일 예측 반복, OHLCV, MinMax, LR)

In [ ]:
train.dtypes

날짜      datetime64[ns]
시가               int32
고가               int32
저가               int32
종가               int32
거래량              int32
거래대금             int64
등락률            float32
종목코드            object
dtype: object



---



---


### ::: 해결 해야 할 사항1 : **학습기간** 120일 학습하고 건너뛰는 방식이 아니라 어떻게 바꿀 수 있는지 방법 찾아야함

---



---



In [ ]:
# ############################################
# ###### 학습시킬 기간과 예측 기간 확인 ######
# ############################################

# train_start_date = train['날짜'].min()
# train_end_date = train_start_date + pd.DateOffset(days=120)
# test_start_date = train_end_date + pd.DateOffset(days=1)
# test_end_date = test_start_date + pd.DateOffset(days=14)

# print(train_start_date)
# print(train_end_date)
# print(test_start_date)
# print(test_end_date)

# print("-----다음구간-----")
# # 다음 구간 설정
# train_start_date += pd.DateOffset(days=120)
# train_end_date += pd.DateOffset(days=120)
# test_start_date += pd.DateOffset(days=120)
# test_end_date += pd.DateOffset(days=120)

# print(train_start_date)
# print(train_end_date)
# print(test_start_date)
# print(test_end_date)

2021-06-01 00:00:00
2021-09-29 00:00:00
2021-09-30 00:00:00
2021-10-14 00:00:00
-----다음구간-----
2021-09-29 00:00:00
2022-01-27 00:00:00
2022-01-28 00:00:00
2022-02-11 00:00:00


In [ ]:
# 특성 변환기 클래스 구현
class FeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_transformed = X.copy()
        X_transformed[self.columns] = X_transformed[self.columns].astype(np.float32)
        return X_transformed

# Pipeline 생성
pipeline = Pipeline([
    ('preprocess', ColumnTransformer(transformers=[('num', FeatureTransformer(['시가', '고가', '저가', '거래량','거래대금'])),], remainder='passthrough')),
    ('scaling', MinMaxScaler()),
    ('model', LinearRegression())
])

# 반복적으로 학습 및 예측 수행
train_start_date = train['날짜'].min()
train_end_date = train_start_date + pd.DateOffset(days=120)
test_start_date = train_end_date + pd.DateOffset(days=1)
test_end_date = test_start_date + pd.DateOffset(days=14)

# 변수 초기화
total_return = 0.0
num_predictions = 0

# 결과를 저장할 DataFrame 생성
results_df = pd.DataFrame(columns=['종목코드', '절대값', '차이'])

while test_end_date <= pd.to_datetime('2023-06-21'):  # 최종 예측일까지 반복
    # 학습 데이터 설정
    train_data = train[(train['날짜'] >= train_start_date) & (train['날짜'] <= train_end_date)]
    features = train_data.drop(['종가'], axis=1)
    labels = train_data['종가']

    # 날짜 데이터를 숫자형으로 변환
    features['날짜'] = features['날짜'].dt.strftime('%Y%m%d').astype(np.int32)

    # 스케일링
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)

    # 모델 학습
    model = LinearRegression()
    model.fit(features_scaled, labels)

    # 예측 데이터 설정
    test_data = train[(train['날짜'] >= test_start_date) & (train['날짜'] <= test_end_date)]
    test_features = test_data.drop(['종가'], axis=1)

    # 날짜 데이터를 숫자형으로 변환
    test_features['날짜'] = test_features['날짜'].dt.strftime('%Y%m%d').astype(np.int32)

    # 스케일링
    test_features_scaled = scaler.transform(test_features)

    # 예측
    predictions = model.predict(test_features_scaled)

    # 종목 코드별로 결과 저장
    for i, pred in enumerate(predictions):
        actual_close = test_data.iloc[i]["종가"]
        print(f'일자: {test_data.iloc[i]["날짜"]}, 종목 코드: {test_data.iloc[i]["종목코드"]}, 예측 종가: {pred}, 실제 종가: {actual_close}')
        if actual_close != 0:
            return_rate = (pred - actual_close) / actual_close
            total_return += return_rate
            num_predictions += 1

            # 결과를 DataFrame에 추가 (종목 코드별로 하나씩)
            if test_data.iloc[i]["종목코드"] not in results_df['종목코드'].values:
                results_df = results_df.append({'날짜': test_data.iloc[i]["날짜"], '종목코드': test_data.iloc[i]["종목코드"], '절대값': abs(return_rate), '차이': return_rate}, ignore_index=True)
            else:
                results_df.loc[results_df['종목코드'] == test_data.iloc[i]["종목코드"], '절대값'] += abs(return_rate)
                results_df.loc[results_df['종목코드'] == test_data.iloc[i]["종목코드"], '차이'] += return_rate

    # 다음 구간 설정
    train_start_date += pd.DateOffset(days=120)
    train_end_date += pd.DateOffset(days=120)
    test_start_date += pd.DateOffset(days=120)
    test_end_date += pd.DateOffset(days=120)

# 최종 수익률 계산
final_return = total_return / num_predictions
print("최종 수익률:", final_return)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
일자: 2023-05-24 00:00:00, 종목 코드: 339950, 예측 종가: 2183.1615033507896, 실제 종가: 2030
일자: 2023-05-25 00:00:00, 종목 코드: 339950, 예측 종가: 2136.0002856868396, 실제 종가: 2005
일자: 2023-05-26 00:00:00, 종목 코드: 339950, 예측 종가: 2173.1940352670954, 실제 종가: 2010
일자: 2023-05-30 00:00:00, 종목 코드: 339950, 예측 종가: 2139.6054311425933, 실제 종가: 2000
일자: 2023-05-23 00:00:00, 종목 코드: 099190, 예측 종가: 16955.117672375916, 실제 종가: 16790
일자: 2023-05-24 00:00:00, 종목 코드: 099190, 예측 종가: 17238.291257255794, 실제 종가: 17240
일자: 2023-05-25 00:00:00, 종목 코드: 099190, 예측 종가: 17269.850129623883, 실제 종가: 17060
일자: 2023-05-26 00:00:00, 종목 코드: 099190, 예측 종가: 17233.822866030983, 실제 종가: 17400
일자: 2023-05-30 00:00:00, 종목 코드: 099190, 예측 종가: 17788.01310225965, 실제 종가: 17580
일자: 2023-05-23 00:00:00, 종목 코드: 289010, 예측 종가: 5092.271368043961, 실제 종가: 4960
일자: 2023-05-24 00:00:00, 종목 코드: 289010, 예측 종가: 5109.474504198883, 실제 종가: 4960
일자: 2023-05-25 00:00:00, 종목 코드: 289010, 예측 종가: 5052.288837824442, 실제 종가: 4915
일자: 2023-05-26 



---



---


### ::: 해결 해야 할 사항2 : **차이 기준**으로 순위매기는게 아니라 다른 방법 찾아야함


---



---



In [ ]:
results_df = results_df.sort_values(by='차이', ascending=False)
results_df['종목코드'] = 'A' + results_df['종목코드']
results_df['순위'] = results_df['차이'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
results_df

In [ ]:
results_df

,종목코드,절대값,차이,순위
422,A096350,17.876681,15.938599,1
1625,A015590,15.754635,14.868609,2
213,A089230,15.984938,14.719891,3
811,A004410,26.169933,14.655455,4
1627,A051780,15.649181,14.552089,5
...,...,...,...,...
1059,A263540,46.619452,-46.619452,1996
353,A065560,46.698929,-46.698929,1997
955,A015540,46.856703,-46.856703,1998
1420,A178780,47.085371,-47.085371,1999


In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv')
sample_submission

,종목코드,순위
0,A000020,1
1,A000040,2
2,A000050,3
3,A000070,4
4,A000080,5
...,...,...
1995,A375500,1996
1996,A378850,1997
1997,A383220,1998
1998,A383310,1999


In [ ]:
baseline_submission = sample_submission[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='left')
baseline_submission

,종목코드,순위
0,A000020,980
1,A000040,17
2,A000050,1184
3,A000070,1683
4,A000080,1592
...,...,...
1995,A375500,1674
1996,A378850,405
1997,A383220,1904
1998,A383310,1710


In [ ]:
baseline_submission.to_csv('/content/drive/MyDrive/Colab Notebooks/baseline_submission0712.csv', index=False)